<a href="https://colab.research.google.com/github/csmaster23/machine_learning_principles/blob/main/Auto_Budget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import csv
import time
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
# Connect your google drive that houses the csv files
month_year = "September 2024"
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Budgets/Auto_Budgets/2024/September/'  # Replace with your file path
csv_name = 'Chase_Credit_Sept_2024.CSV'
path = os.path.join(file_path, csv_name)
print(path)

In [ ]:
# Load csv where each row is a dictionary with the appropriate key words for each row from the column names
def load_csv(file_path):
  data = []
  with open(file_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      data.append(row)
  return data
print(f"Loading {path}...")
data = load_csv(path)
print("Finished.")

In [ ]:
# Find all the potential column names from the uploaded csv
keys = data[0].keys()
print(keys)

In [ ]:
# What are the categories you want for spending?
spend_dict = {
    0: ("Groceries - Food",           1000.0),
    1: ("Groceries - House",           100.0),
    2: ("LinkedIn",                     35.0),
    3: ("Savings",                     200.0),
    4: ("Fast Food",                   100.0),
    5: ("Date Nights",                  50.0),
    6: ("iCloud Storage",                5.0),
    7: ("Amazon Web Services",           1.0),
    8: ("Internet",                     60.0),
}
total_planned_spending = sum([spend_dict[key][1] for key in spend_dict.keys()])
print(f"Total planned spending: ${total_planned_spending}")

In [ ]:
# What are the categories you want for earning?
earn_dict = {
    0: ("W-2",                      3000.0),
}
total_planned_earning = sum([earn_dict[key][1] for key in earn_dict.keys()])
print(f"Total planned earning: ${total_planned_earning}")

In [ ]:
def spend_pretty_print(spend_dict):
  # Calculate the max width for each field to align columns
  key_width = max(len(str(key)) for key in spend_dict.keys())
  first_col_width = max(len(str(spend_dict[key][0])) for key in spend_dict.keys())
  second_col_width = max(len(str(spend_dict[key][1])) for key in spend_dict.keys())

  # Define the format string for three columns
  row_format = f"{{:<{key_width}}}  {{:<{first_col_width}}}  {{:<{second_col_width}}}"

  # Print each entry in three columns
  count = 0
  for key in spend_dict.keys():
      print(row_format.format(key, spend_dict[key][0], spend_dict[key][1]), end='    |    ')
      count += 1
      if count % 3 == 0:
          print()  # Move to the next line after three columns

  # If there are any remaining entries that don’t fill a full row, print a newline
  if count % 3 != 0:
      print()

In [ ]:
spend_pretty_print(spend_dict)
display_limit = 5
sorted_spent = {}

# Loop through each row in data
for index, row in enumerate(data):
    post_date = row['Post Date']
    descr = row['Description']
    amount = float(row['Amount'])

    # Only process negative amounts
    if amount < 0:
        # Get user input and categorize amount
        print(f"\n>> Date posted: {post_date} - {descr}: ${amount}")
        custom_key = int(input(f"Which custom category should this go to: "))
        if custom_key not in sorted_spent:
            sorted_spent[custom_key] = []
        sorted_spent[custom_key].append(amount)

        # Brief pause to ensure the user sees the input prompt
        time.sleep(0.1)

        # Clear the output cell
        clear_output(wait=True)
        spend_pretty_print(spend_dict)

        # Display the last 5 responses
        start_index = max(0, index - display_limit + 1)
        print(start_index, " / ", len(data))
        for i in range(start_index, index + 1):
            prev_row = data[i]
            prev_date = prev_row['Post Date']
            prev_descr = prev_row['Description']
            prev_amount = float(prev_row['Amount'])
            if prev_amount < 0:
                print(f"Date posted: {prev_date} - {prev_descr}: ${prev_amount}")


In [ ]:
all_sums, exp_sums = 0, 0
table = []
cats, expected, spent = [], [], []
for key in sorted_spent.keys():
    amount = -1 * sum(sorted_spent[key])
    all_sums += sum(sorted_spent[key])
    exp_sums += (-1 * spend_dict[key][1])
    cats.append(spend_dict[key][0])
    expected.append((spend_dict[key][1]))
    spent.append(amount)
    table.append([spend_dict[key][0], spend_dict[key][1], amount])

column_widths = [max(len(str(row[i])) for row in table) for i in range(3)]
# table.append(["", "Total Expected:", f"${-1 * exp_sums:.2f}", "Total spent:", f"${-1 * all_sums:.2f}"])

headers = ["Category", "Expected", "Spent"]
for i, header in enumerate(headers):
    column_widths[i] = max(column_widths[i], len(header))

header_row = "  ".join(f"{header:<{column_widths[i]}}" for i, header in enumerate(headers))
print(header_row)
print("-" * len(header_row))
for row in table:
    print("  ".join(f"{str(row[i]):<{column_widths[i]}}" for i in range(3)))
print("-----------------------------------------------")
print("")
print(f"Total Expected Expenditure:    ${sum(expected):.2f}")
print(f"Total Expenditure:             ${sum(spent):.2f}")
print(f"Difference:                    ${sum(expected) - sum(spent):.2f}")

In [ ]:
# Bar Chart - Expected vs Actual Spending
x = np.arange(len(cats))  # label locations
width = 0.35  # bar width

fig, ax = plt.subplots(figsize=(12, 8))
bars1 = ax.bar(x - width/2, expected, width, label='Expected')
bars2 = ax.bar(x + width/2, spent, width, label='Spent')

# Labels and title
ax.set_xlabel('Categories', fontsize=14)
ax.set_ylabel('Amount ($)', fontsize=14)
ax.set_title('Budget vs Actual Spending by Category', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels(cats, fontsize=12, rotation=45, ha='right')
ax.legend(fontsize=12)

# Add value labels with padding
for bar in bars1 + bars2:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, yval + 20, f"${yval:.0f}", ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Pie Chart - Percentage of Total Expected vs. Spent
fig, ax = plt.subplots(1, 2, figsize=(14, 7))

# Pie chart for Expected
ax[0].pie(expected, labels=cats, autopct='%1.1f%%', startangle=140, wedgeprops={'edgecolor': 'black', 'linewidth': 1})
ax[0].set_title("Expected Spending Distribution", fontsize=14)

# Pie chart for Spent
ax[1].pie(spent, labels=cats, autopct='%1.1f%%', startangle=140, wedgeprops={'edgecolor': 'black', 'linewidth': 1})
ax[1].set_title("Actual Spending Distribution", fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
this_month_expected = sum(expected)
this_month_spent = sum(spent)
this_month_diff = this_month_expected - this_month_spent
print(f"This month expected:   ${this_month_expected:.2f}")
print(f"This month spent:      ${this_month_spent:.2f}")
print(f"This month difference: ${this_month_diff:.2f}")

In [ ]:
# 2024 Totals - Must manually update this on your own
months = ['Sept']                # List of months
monthly_expected = [1915.00]     # Monthly budgeted totals
monthly_spent = [3513.49]        # Monthly actual spending totals

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

diff = np.array(monthly_spent) - np.array(monthly_expected)
colors = ['green' if d <= 0 else 'red' for d in diff]

bars = ax.bar(months, monthly_spent, color=colors, label="Spent")
ax.plot(months, monthly_expected, marker='o', color='blue', label="Budgeted", linestyle='--')

# Add data labels
for bar, d in zip(bars, diff):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f"${int(bar.get_height())}",
            ha='center', va='bottom' if d >= 0 else 'top', color="black", fontsize=10)

# Labels and title
ax.set_xlabel("Month", fontsize=14)
ax.set_ylabel("Amount ($)", fontsize=14)
ax.set_title("Monthly Spending with Over-Budget Highlights", fontsize=16)
ax.legend()

plt.tight_layout()
plt.show()